In [1]:
import os
import pandas as pd
from pathlib import Path

if not "root" in locals():
    current_path = Path(os.getcwd())
    root = current_path.parent.absolute()

In [2]:
os.chdir(root)
from data_treatment.box_creator import create_binary_box

df = pd.read_csv('database.csv')
df = create_binary_box(df, threshold=2., box_length=7)

In [9]:
df[["Bullish_SENTIMENT", "Bearish_SENTIMENT", "Neutral_SENTIMENT"]]

,Bullish_SENTIMENT,Bearish_SENTIMENT,Neutral_SENTIMENT
0,0.600000,0.150000,0.250000
1,0.600000,0.150000,0.250000
2,0.600000,0.150000,0.250000
3,0.750000,0.133300,0.116600
4,0.750000,0.133300,0.116600
...,...,...,...
5377,0.365188,0.269625,0.365188
5378,0.365188,0.269625,0.365188
5379,0.370253,0.262658,0.367089
5380,0.370253,0.262658,0.367089


**1er modèle** : Sélection à la main des features

In [12]:
cols = ["PX_LAST_VIX_volatility",
        "VOLUME_TOTAL_CALL_VIX_volatility",
        "VOLUME_TOTAL_PUT_VIX_volatility",
        "PX_LAST_SPX_volatility",
        "PX_VOLUME_SPX_volatility",
        "PX_LAST_VSTOXX_volatility",
        "Bullish_SENTIMENT",
        "Neutral_SENTIMENT"]

hand_picked_df = df.set_index("Date")[cols + ["Box"]].dropna(axis = 0)

Essai avec régression logistique

In [13]:
import statsmodels.api as sm

X = hand_picked_df.drop(columns="Box")
y = hand_picked_df["Box"]
scaled_X = sm.add_constant((X - X.mean())/X.std())

logit_model=sm.Logit(y,scaled_X)
result=logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.609552
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                    Box   No. Observations:                 3788
Model:                          Logit   Df Residuals:                     3779
Method:                           MLE   Df Model:                            8
Date:                Wed, 05 Jan 2022   Pseudo R-squ.:                 0.04107
Time:                        12:29:26   Log-Likelihood:                -2309.0
converged:                       True   LL-Null:                       -2407.9
Covariance Type:            nonrobust   LLR p-value:                 1.882e-38
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -0.7204      0.036    -20.27